In [ ]:
import torch
print(torch.cuda.is_available())  # True가 출력되어야 합니다.
print(torch.cuda.device_count())  # 사용 가능한 GPU 개수가 출력됩니다.


In [4]:
import os
import shutil
import random

def filter_and_save_by_index(input_image_dir, input_label_dir, output_dir, allowed_indices, num_files_to_save):
    """
    Filter files by specific class indices and save them to a new directory.

    Args:
        input_image_dir (str): Path to the directory containing image files.
        input_label_dir (str): Path to the directory containing label (.txt) files.
        output_dir (str): Path to the output directory where filtered files will be saved.
        allowed_indices (list): List of allowed class indices to filter.
        num_files_to_save (int): Number of random files to save.
    """
    # Create output directories
    output_image_dir = os.path.join(output_dir, 'images')
    output_label_dir = os.path.join(output_dir, 'labels')
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    # Filter label files by allowed indices
    filtered_files = []
    for label_file in os.listdir(input_label_dir):
        if label_file.endswith('.txt'):
            label_path = os.path.join(input_label_dir, label_file)
            with open(label_path, 'r') as f:
                lines = f.readlines()
                # Filter lines by allowed indices
                filtered_lines = [line for line in lines if int(line.split()[0]) in allowed_indices]

            # If any valid lines remain, save the file and associate it with the corresponding image
            if filtered_lines:
                base_name = os.path.splitext(label_file)[0]
                image_file = base_name + '.jpg'  # Assuming images are .jpg
                if os.path.exists(os.path.join(input_image_dir, image_file)):
                    filtered_files.append((label_file, filtered_lines, image_file))

    # Randomly select the specified number of files to save
    selected_files = random.sample(filtered_files, min(num_files_to_save, len(filtered_files)))

    # Save selected files to the output directory
    for label_file, filtered_lines, image_file in selected_files:
        # Save filtered label file
        output_label_path = os.path.join(output_label_dir, label_file)
        with open(output_label_path, 'w') as f:
            f.writelines(filtered_lines)

        # Copy the corresponding image file
        shutil.copy(os.path.join(input_image_dir, image_file), os.path.join(output_image_dir, image_file))

# Example usage
input_image_dir = r"C:\\Users\\KJJ\\Desktop\\code\\data\\temp\\train_origin\\images"
input_label_dir = r"C:\\Users\\KJJ\\Desktop\\code\\data\\temp\\train_origin\\labels"
output_dir = r"C:\\Users\\KJJ\\Desktop\\code\\data\\temp\\train"
allowed_indices = [3]  # Only these indices will be kept
num_files_to_save = 300  # Number of random files to save

filter_and_save_by_index(input_image_dir, input_label_dir, output_dir, allowed_indices, num_files_to_save)

In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 객체 생성
model = YOLO('yolov8s.pt')

# 모델 학습
model.train(
    data="/home/kjj73/dev_ws/yolo/yolov8/data/dataset_003/data.yaml",  # Raw 문자열 사용
    epochs=200,
    imgsz=(320, 640),
    batch=16,
    device=0,
    # num_workers=4,
    mosaic=1.0,
    mixup=0.2,
    degrees=20,
    translate=0.2,
    scale=0.7,
    shear=5.0,
    perspective=0.001,
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.5,
    fliplr=0.5,
    patience=30,
    conf=0.6,
    iou=0.6,
    # workers=4,
    project="./",
    save=True
)

# 학습 완료 후 결과 확인
results = model.val(
    data="/home/kjj73/dev_ws/yolo/yolov8/data/dataset_003/data.yaml", # Raw 문자열 사용
    imgsz=640,
    iou=0.55,
    save_json=True,
    project="./"
)

# 학습 및 검증 결과 출력
print("Validation Results:", results)


In [4]:
import cv2
from ultralytics import YOLO

# 학습된 모델 파일 경로
MODEL_PATH = "/home/kjj73/dev_ws/yolo/yolov8/train/weights/best.pt" # 자신의 모델 경로로 수정

# YOLOv8 모델 로드
model = YOLO(MODEL_PATH)

# 카메라 캡처 초기화
cap = cv2.VideoCapture(0)  # 기본 카메라 사용 (0번 장치)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

# 탐지 설정
CONFIDENCE_THRESHOLD = 0.5  # 탐지 최소 신뢰도

# 클래스 색상 정의 (랜덤)
import random
random.seed(42)
COLORS = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(len(model.names))]

# 실시간 탐지 루프
while True:
    ret, frame = cap.read()
    if not ret:
        print("카메라에서 프레임을 읽을 수 없습니다.")
        break

    # 모델을 사용하여 탐지 수행
    results = model.predict(
        source=frame, 
        conf=0.5, 
        verbose=False,
        device=0, 
        imgsz=640
    )

    # 탐지 결과를 이미지에 표시
    detections = results[0].boxes.data.cpu().numpy()  # 탐지 결과 데이터
    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection
        class_id = int(class_id)
        label = f"{model.names[class_id]} {confidence:.2f}"

        # 박스 및 라벨 표시
        color = COLORS[class_id % len(COLORS)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 화면에 출력
    cv2.imshow("YOLOv8 Detection", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 정리
cap.release()
cv2.destroyAllWindows()


In [ ]:
import torch

# 모델 로드
model = torch.load("/home/kjj73/dev_ws/yolo/yolov8/data/traffic_light_best.pt")
model.eval()  # 학습 모드로 전환 시 model.train() 사용


In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 객체 생성
# 모델 로드
model = YOLO("/home/kjj73/dev_ws/yolo/yolov8/data/traffic_light_best.pt")
# model.eval()  # 학습 모드로 전환 시 model.train() 사용

# 모델 학습
model.train(
    data="/home/kjj73/dev_ws/yolo/yolov8/data/dataset_002/data.yaml",  # Raw 문자열 사용
    epochs=60,
    imgsz=(320, 640),
    batch=16,
    device=0,
    # num_workers=4,
    mosaic=1.0,
    mixup=0.2,
    degrees=20,
    translate=0.2,
    scale=0.7,
    shear=5.0,
    perspective=0.001,
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.5,
    fliplr=0.5,
    # patience=20,
    conf=0.6,
    iou=0.6,
    # workers=4,
    project="./",
    save=True
)

# 학습 완료 후 결과 확인
results = model.val(
    data="/home/kjj73/dev_ws/yolo/yolov8/data/dataset_002/data.yaml", # Raw 문자열 사용
    imgsz=640,
    iou=0.55,
    save_json=True,
    project="./"
)

# 학습 및 검증 결과 출력
print("Validation Results:", results)
